# RELANCEZ BIEN LE NOTEBOOK ENTIER A CHAQUE TRAINING POUR GARDER UNE TRACE DES RESULTATS DE CHAQUE MODEL

le model.summary est saved dans saved_models/nom_du_run_wandb

In [1]:
import sys
sys.path.append('../data_cleaning')

from time_series_forecasting import create_dataset
from dataset import build_dataset

In [2]:
import wandb
import tensorflow as tf

In [3]:
from tensorflow.keras.layers import Dense
from keras.layers import Input,Conv1D,MaxPooling1D,Bidirectional,Dropout,CuDNNLSTM,Dense,Input,LSTM, LeakyReLU
from tensorflow.keras import Sequential

### create wandb_api_key.txt and paste your api key inside &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; from https://wandb.ai/settings

In [4]:
file = open('wandb_api_key.txt', 'r')
wandb.login(key=file.readline()) # mettez
file.close()
            

wandb: Currently logged in as: duramann (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\theo-/.netrc


In [5]:
wandb.init(project="AI_Based_Sensor_Information_Fusion", entity="efrai")
batch_size = 128
epochs = 200
learning_rate = 1e-5
##To edit: 
config = {
    "learning_rate": learning_rate,
    "epochs": epochs,
    "batch_size": batch_size,
    "window_size": 30,
    "stride":5,
    "number_imu":3,
    "bidirectional":True,
    "lstm_number": 2,
    "lstm_neurons":128,
    "dropout":0.2,
    "dense_number":2,
    "dense_first_neuron":128,#puissance de 2
    "activation_dense":'relu',
}

wandb.init(project="AI_Based_Sensor_Information_Fusion", entity="efrai",config=config)

In [6]:
x_train, y_train = build_dataset(0, 50, wandb.config['window_size'],wandb.config['stride'],wandb.config['number_imu'])
x_test, y_test = build_dataset(51,60, wandb.config['window_size'],wandb.config['stride'],wandb.config['number_imu']) 

In [7]:
model = Sequential()
model.add(Input((wandb.config['window_size'], 6*wandb.config['number_imu'])))
if wandb.config['bidirectional']:
    model.add(Bidirectional(CuDNNLSTM(wandb.config['lstm_neurons'], return_sequences=True)))
else:
    model.add(LSTM(wandb.config['lstm_neurons'], return_sequences=True))
for s in range(wandb.config['lstm_number']-1):
    if wandb.config['bidirectional']:
        model.add(Bidirectional(CuDNNLSTM(wandb.config['lstm_neurons'])))
    else:
        model.add(CuDNNLSTM(wandb.config['lstm_neurons']))
model.add(Dropout(wandb.config['dropout']))
for d in range(wandb.config['dense_number']):
    model.add(Dense(int(wandb.config['dense_first_neuron'])/(d+1), activation=wandb.config['activation_dense']))
model.add(Dense(6, activation='relu'))
    
    

opt = tf.keras.optimizers.Adam(learning_rate)
model.compile(optimizer=opt, loss='mse')  

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 30, 256)          151552    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 256)              395264    
 nal)                                                            
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 6)                 3

In [9]:
model.fit(
    x_train,
    y_train,
    validation_split=0.1,
    batch_size = wandb.config['batch_size'],
    epochs=wandb.config['epochs'],
    callbacks=[wandb.keras.WandbCallback(log_batch_frequency=5)])

Epoch 1/200
350/350 [==============================] - 12s 24ms/step - loss: 0.1808 - _timestamp: 1648140509.3803 - _runtime: 30.3803 - val_loss: 0.0980
Epoch 2/200
350/350 [==============================] - 8s 22ms/step - loss: 0.0958 - _timestamp: 1648140517.8451 - _runtime: 38.8451 - val_loss: 0.0882
Epoch 3/200
350/350 [==============================] - 8s 22ms/step - loss: 0.0892 - _timestamp: 1648140525.6338 - _runtime: 46.6338 - val_loss: 0.0855
Epoch 4/200
350/350 [==============================] - 8s 22ms/step - loss: 0.0866 - _timestamp: 1648140533.5070 - _runtime: 54.5070 - val_loss: 0.0843
Epoch 5/200
350/350 [==============================] - 8s 23ms/step - loss: 0.0850 - _timestamp: 1648140541.4507 - _runtime: 62.4507 - val_loss: 0.0831 - _timestamp: 1648140541.0000 - _run
Epoch 6/200
350/350 [==============================] - 8s 23ms/step - loss: 0.0839 - _timestamp: 1648140549.4366 - _runtime: 70.4366 - val_loss: 0.0823
Epoch 7/200
350/350 [=============================

In [12]:
wandb.finish()

In [46]:
y_pred = model.predict(x_test)

In [47]:
from sklearn.metrics import r2_score

for i in range(6):
    print(r2_score(y_test[:][i], y_pred[:][i]))

0.6461840805004742
0.9724071918961045
0.8537899439979582
0.92304257672397
0.9292223710253416
0.9865244926496125
